### Goal: Solve the issue by using langchain and start a pull request

In [2]:
import re
from dotenv import load_dotenv
from os import getenv

from autocoder.database.models import Repository
from autocoder.langchain_shortcuts import (
    get_response,
    get_file_paths,
    get_code_response,
    extract_files,
)

load_dotenv()


issue_url = "https://github.com/shroominic/VoiceGPT/issues/21"
repo_url, issue_number = issue_url.split("/issues/")

repo = Repository(repo_url, getenv("GITHUB_TOKEN"))
issue = repo.get_issue(issue_number)

A branch named 'issue-21' already exists. Switching to the existing branch.


#### Init prompt process

In [3]:
context = {
    "repo_name": issue.repository.name, 
    "repo_keywords": issue.repository.keywords, 
    "repo_description": issue.repository.description, 
    "issue_title": issue.title, 
    "issue_number": str(issue.issue_number),
    "issue_description": issue.body, 
    "code_tree": issue.repository.codebase.tree
}

coding_system_prompt = """
You are a coding assistant solving tasks for developers.
Tasks are represented as issues on GitHub repositories.
Be verbose and precise in your responses.

The following is a conversation about solving issue {issue_title} #{issue_number} on {repo_name}.
"""


#### Collect important files to gain context

In [4]:
important_files_prompt = """
Which files are important to read for gaining understanding of the codebase?

Codebase: 
{code_tree}

For example in a python project you might want to look at the 'requirements.txt' file to understand which technologies are used in the project.\n
Reply like this and make sure pathes are in 'single quotes' and not "double quotes":

```python
relevant_files = [
    'path/to/file1.txt',  # do not start with ./ or /
    'path/to/file2.js',
    ...
    # max 4 files so choose wisely
]
```
"""

relevant_file_paths = await get_file_paths(
    important_files_prompt, 
    "relevant_files",
    system_instruction=coding_system_prompt,
    validation=issue.codebase.validate_file_paths,
    **context
)

relevant_files_dict = {}
for path in relevant_file_paths:
    path = path.strip("'") # TODO: remove this 
    relevant_files_dict[path] = issue.codebase.show_file(path)

context["relevant_files"] = str(relevant_files_dict)
relevant_files_dict

autocoder/.codebases/VoiceGPT/src/__main__.py
autocoder/.codebases/VoiceGPT/src/web/main.py
autocoder/.codebases/VoiceGPT/src/bot/sessions/__init__.py
autocoder/.codebases/VoiceGPT/src/bot/cogs/__init__.py


{'src/__main__.py': 'import argparse\n\n\ndef main():\n    parser = argparse.ArgumentParser(description=\'Choose which app to start\')\n    parser.add_argument(\'app\', choices=[\'bot\', \'web\'], help=\'The app to start\')\n    args = parser.parse_args()\n    \n    if args.app == \'bot\':\n        import bot\n        bot.main()\n\n    elif args.app == \'web\':\n        import uvicorn\n        from web import main\n        uvicorn.run(main.app, host="0.0.0.0", port=8000)\n    \n\nif __name__ == \'__main__\':\n    main()\n',
 'src/web/main.py': 'from fastapi import FastAPI\n\nfrom fastapi_users import FastAPIUsers, models\nfrom fastapi_users.authentication import OAuth2CookieBearer\nfrom fastapi_users.db import SQLAlchemyUserDatabase\nfrom typing import Optionalfrom fastapi.staticfiles import StaticFiles\n\nfrom web.database.models.user import User\nfrom web.config import setup_db, setup_authentication\nfrom web.oauth import on_after_register, on_after_authenticatefrom web.endpoints imp

Generate repository summary/context:

In [5]:
repo_summary_prompt = """
Summarize what this repository is about and what it does to have a better understanding and context of the codebase.
Repository name:
{repo_name}
Repository description:
{repo_description}
Repository keywords:
{repo_keywords}
Repository codebase:
{code_tree}
Relevant files: 
{relevant_files}

Describe the technologies used and the structure of the codebase.
Please be as detailed and precise as possible but keep it short.
"""

context["repo_summary"] = await get_response(
    repo_summary_prompt, 
    system_instruction=coding_system_prompt, 
    **context
)

Generate issue summary/context and decide which files to change and create:

In [6]:
issue_summary_prompt = """
Repository codebase:
{code_tree}
Repository Summary:
{repo_summary}
Issue title: 
{issue_title}
Issue description: 
{issue_description}

Describe step by step (abstract) how to implement the issue and what files are relevant.
Be precise and keep it short.
"""

get_changes_prompt = """
Repository codebase:
{code_tree}
Repository description:
{repo_summary}
Issue description:
{issue_summary}

What files need to be changed to solve the issue?
What new files need to be created?

Write paths using 'single quotes'.
Make sure to put only file paths that exist in the codebase/project directory in files_to_change.
Respond with a codeblock like this:
```python
new_files = [
    'path/to/new_file1.py', # do not start with ./ or /
    'path/to/new_file2.ipynb',
    ...
]

files_to_change = [
    'path/to/file1.txt',  # do not start with ./ or /
    'path/to/file2.js',
    ...
]
```
"""

async def get_files(retry=3):
    try:
        context["issue_summary"] = await get_response(
            issue_summary_prompt,
            system_instruction=coding_system_prompt,
            verbose=True,
            **context
        )
        
        files_to_change = await get_response(
            get_changes_prompt,
            system_instruction=coding_system_prompt,
            verbose=True,
            **context
        )

        new_files = extract_files(files_to_change, "new_files")
        files_to_change = extract_files(files_to_change, "files_to_change")

        print("New files:", new_files)
        print("Files to change:", files_to_change)
        if not issue.codebase.validate_file_paths(files_to_change):
            raise ValueError("Some files to change do not exist in the codebase.")

        if any(issue.codebase.file_exists(f) for f in new_files): 
            raise ValueError("Some new files already exist in the codebase.")

        return new_files, files_to_change
    
    except Exception as e:
        if retry > 0:
            print("Retrying...", e)
            return await get_files(retry=retry-1)
        else: raise e


new_files, files_to_change = await get_files()

print("New files:", new_files)
print("Files to change:", files_to_change)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a coding assistant solving tasks for developers.
Tasks are represented as issues on GitHub repositories.
Be verbose and precise in your responses.

The following is a conversation about solving issue /feedback commad to get feedback from users #21 on VoiceGPT.

Human: 
Repository codebase:
./README.md
./.gitignore
./.env.example
.vscode/launch.json
src/__init__.py
src/__main__.py
src/database/alembic.ini
src/database/__init__.py
src/database/models/user.py
src/database/models/session.py
src/database/models/__init__.py
src/database/models/vote.py
src/database/alembic/script.py.mako
src/database/alembic/env.py
src/database/alembic/README
src/database/alembic/versions/9b9db578e8fb_add_vote_model.py
src/database/alembic/versions/644bdf304451_created_at_and_chat_history.py
src/web/requirements.txt
src/web/__init__.py
src/web/main.py
src/web/endpoints/legal.py
src/web/endpoints/home.py
src/web/endpoints/__init__.py


Create new files and change already exsisting ones

In [7]:
new_file_prompt = """
Repository description:
{repo_summary}

Issue description:
{issue_summary}

Implement this file {file_path} to solve the issue.
After completion this the file will be created and added to the codebase.
Reply with a codeblock containing the content of the new file.
"""

new_files_dict = {}
for path in new_files:
    context["file_path"] = path
    
    new_file_content = await get_code_response(
        new_file_prompt,
        system_instruction=coding_system_prompt,
        **context
    )
    
    new_files_dict[path] = new_file_content



change_file = """
Repository description:
{repo_summary}
Issue description:
{issue_summary}
First write a detailed instruction on how to change the file {file_path}.

# [Begin file content]
{file_content}
# [Eind file content]

"""
"""

Then create a list called "changes".
This list should contain tuples of the form (line_number, action, "new code").
Action can be one of the following:
- add: insert a new line of code at the line with the given number and shift all following lines down
- overwrite: overwrite the line with the given number
- delete: delete the line with the given number

Write down all changes and 
reply with a codeblock like this:
```python
# (line_number, 'action', 'new code')
# make sure use 'single quotes' and not "double quotes" or `backticks` like in this example:
changes = [
    (0, 'add', 'import newexample'),
    (4, 'overwrite', 'def foo():'),
    (5, 'add', '    print("hello world")'),
    (6, 'delete', '    print("bar")'),
    ...
]
```
"""

change_files_dict = {}
for path in files_to_change:
    context["file_path"] = path
    context["file_content"] = issue.codebase.show_file(path)
    
    changes_content = await get_code_response(
        change_file,
        system_instruction=coding_system_prompt,
        **context
    )
    
    tuple_pattern = re.compile(r"\((\d+),\s*\'(.*?)\',\s*\'(.*?)\'\)", re.DOTALL)
    tuples = tuple_pattern.findall(changes_content)
    
    change_files_dict[path] = [
        (int(line), action, change) 
        for line, action, change in tuples
    ]


In [8]:
# TODO: Format code using chatgpt 
# and check if content is the same as the original file

In [9]:
for file, content in new_files_dict.items():
    issue.codebase.create_file(file, content)
    print("Created:", file, "\nwith content:", content, "\n")
    
for file, changes in change_files_dict.items():
    issue.codebase.change_file(file, changes)
    print("Changed:", file, "\nwith changes:", changes, "\n")

Created: src/bot/cogs/commands/feedback.py 
with content: from discord.ext import commands
from discord.ext.commands import Cog, Context

class Feedback(Cog):
    def __init__(self, bot):
        self.bot = bot

    @commands.command(name="feedback", brief="Submit feedback about the VoiceGPT bot")
    async def feedback(self, ctx: Context, *, feedback_text: str):
        # Process the user's feedback and store it, or send it to a predetermined channel
        # You can replace CHANNEL_ID with the actual channel ID you want to send the feedback to
        channel = self.bot.get_channel(CHANNEL_ID)
        await channel.send(feedback_text)

        response_message = "Thank you! We appreciate your feedback."
        await ctx.send(response_message)

def setup(bot):
    bot.add_cog(Feedback(bot)) 

Changed: src/bot/cogs/commands/__init__.py 
with changes: [] 



reformatted autocoder/.codebases/VoiceGPT/src/bot/cogs/commands/feedback.py
All done! ✨ 🍰 ✨
1 file reformatted.
All done! ✨ 🍰 ✨
1 file left unchanged.


In [10]:
issue.codebase.commit_changes("Solved issue using GPT-4")

In [11]:
# TODO: Test code and repeat process if necessary

In [12]:
# TODO: Create a pull request

Idea: write code for def change_file(filename, instructions: str, selected_instruction: int) -> None;

iterate over the file and change code step by step

always only implenment a single instruction and tell the bot that previouse instructions already got implemented

do implementation using an agent so the agent can look through documentation or web while coding but not everything is automated by the agent so it is mixed scripting + agent